In [13]:
from dotenv import load_dotenv
import os
from langchain.document_loaders import CSVLoader
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [14]:
load_dotenv()

True

In [21]:
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()

In [19]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

In [23]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(documents, embedding_function)
query = "How many quantity of the wallet is left?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Name: TRAVEL CARD WALLET SKULLS
Quantity: 24
UnitPrice: 0.36


In [20]:

loader = CSVLoader("./e-commerce-data.csv")
documents = loader.load()

db = Chroma.from_documents(documents, embeddings)
query = "How many quantity of the wallet is left?"
docs = db.similarity_search(query)
print(docs[0].page_content)

KeyboardInterrupt: 

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)
chat.openai_api_key = os.getenv("OPENAI_API_KEY")

result = chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)
print(result.content)

chat.openai_api_key = os.getenv("OPENAI_API_KEY")